In [33]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

#Mantengo la terminología de movies y credits para que vayan acorde a los links

In [3]:
print(movies.columns)
print(credits.columns)

#Me he impreso estas dos porque estoy teniendo un problema a la hora de mergear los
#dos df y creo que puede tener que ver con los títulos

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')
Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


In [4]:
#A ver ahora
df_combinado = movies.merge(credits, on='title')
print(df_combinado.columns)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')


In [5]:
#Perfecto, ahora ya puedo pasar a quedarme solamente con las columnas que indica el
#enunciado: movie_id, title, overview, genres, keywords, cast, crew

df_combinado = df_combinado[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [6]:
#Ahora convierto desde json. Este es uno de los pasos que más tiempo me ha llevado.
#Al final he optado por guardar los valores procesados en lista_generos, pasar de json
#a diccionario de python, sacar los name del json y combinarlos

lista_generos = []
for entry in df_combinado["genres"]:
    try:
        items = json.loads(entry)
        names = [item["name"] for item in items]
        lista_generos.append(" ".join(names))
    except:
        lista_generos.append("")

df_combinado["genres"] = lista_generos

#Por otro lado proceso la lista_keywords exactamente de la misma manera, convierto a
#python, selecciono keywords y combino
lista_keywords = []
for entry in df_combinado["keywords"]:
    try:
        items = json.loads(entry)
        names = [item["name"] for item in items]
        lista_keywords.append(" ".join(names))
    except:
        lista_keywords.append("")

df_combinado["keywords"] = lista_keywords


In [7]:
#Ahora hago literamente lo mismo con cast. La única diferencia es que para este nos
#piden que sean solamente los tres primeros, en lugar de todos, por eso añado [:3]

lista_cast = []
for entry in df_combinado["cast"]:
    try:
        items = json.loads(entry)
        names = [item["name"] for item in items[:3]] #<--- Aquí está lo de los 3 primeros
        lista_cast.append(" ".join(names))
    except:
        lista_cast.append("")

df_combinado["cast"] = lista_cast

In [8]:
#Tiene que haber una forma más rápida de hacer esto. Creo que tardaría más en encontrarla
#de lo que tardo en cambiar cuatro cosas del código anterior para hacer lo mismo,
#esta vez en crew

lista_crew = []
for entry in df_combinado["crew"]:
    try:
        items = json.loads(entry)
        nombre_director = ""
        for item in items:
            if item["job"].lower() == "director":
                nombre_director = item["name"]
                break
        lista_crew.append(nombre_director)
    except:
        lista_crew.append("")

df_combinado["crew"] = lista_crew

#Si antes lo digo, antes me toca rectificarlo. He tenido que meter un bucle for para sacar
#el nombre del director. El resto del código es igual.

In [9]:
#Overview es distinta porque hay que convertirla en una lista de palabras y limpiarla
#Es decir, asegurarme de que es texto, quitarle espacios, pasarlo a minúsculas etc

overview_list = []
for text in df_combinado["overview"]:
    if isinstance(text, str):
        overview_list.append(text.split())
    else:
        overview_list.append([])

df_combinado["overview"] = overview_list

#Ahora les quito los espacios y las pongo en minúsculas
for column in ["genres", "keywords", "cast", "crew"]:
    df_combinado[column] = df_combinado[column].str.replace(" ", "").str.lower()

In [10]:
#Este paso es el que más me ha chocado y el que voy a recordar para futuras ocasiones
#cuando use KNN. Ahora hay que combinarlo todo en UN SOLO TAG

#Qué sentido tiene esto?
#Hasta ahora estoy acostumbrado a manetener las variables separadas para ver cuánto
#valor predictivo tiene cada una, pero en este caso hay que pasarlo todo a un solo tag
#El modelo del KNN necesita ver en un solo bloque de texto toda la información y compara
#a partir de ahí

#Tiene pinta de que este modelo está definiendo cada palabra como si fuera una dimensión
#que puede estar presente o no, seguramente lo convierta en una matriz que podría a su vez
#representarse como hipercubo donde cada palabra es un vértice, y luego irá recorriendo las
#aristas del hipercubo para pasar de un vértice a otro y ver cuántos pasos le lleva. Cuanto
#más cerca queden, más parecidas serán según el algoritmo.
#Lo que no creo es que el algoritmo utilice la distancia euclidiana para medir la distancia

#Efectivamente, he estado investigando y usan la distancia coseno (mide el ángulo entre
#los vectores). Al final la idea de base es la misma, contar saltos.

#Me gusta mucho este algoritmo y creo que puede resultar en agrupamientos muy útiles.
#Lo único que no sé si se está aplicando o no es algún criterio para determinar qué dos
#películas se perciben como más cercanas aunque aparezcan a la misma distancia coseno.
#No es necesario complicarse tanto para este ejercicio de todos modos

#Aquí está la lista tags en la que se incluye todo

lista_tags = []
for i in range(len(df_combinado)):

    tags = (
        " ".join(df_combinado["overview"][i])
        + " " +
        df_combinado["genres"][i]
        + " " +
        df_combinado["keywords"][i]
        + " " +
        df_combinado["cast"][i]
        + " " +
        df_combinado["crew"][i]
    )
    lista_tags.append(tags)

df_combinado["tags"] = lista_tags

In [11]:
# Mostrar las primeras filas del DataFrame transformado
print(df_combinado[["title", "tags"]].head())

#Lo que pensaba que sería simplemente mostrar los avances hasta ahora ha resultado ser
#mostrar los fallos hasta ahora.
#Aparecen todas las palabras juntas. Toca separarlas

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                                tags  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...  


In [18]:
# Asegurarnos de que las columnas son cadenas de texto
df_combinado["genres"] = df_combinado["genres"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
df_combinado["keywords"] = df_combinado["keywords"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
df_combinado["cast"] = df_combinado["cast"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
df_combinado["crew"] = df_combinado["crew"].apply(lambda x: str(x))

# Combinar todas las columnas transformadas en una sola columna llamada 'tags'
df_combinado["tags"] = (
    df_combinado["overview"].apply(lambda x: " ".join(x)) + " " +
    df_combinado["genres"] + " " +
    df_combinado["keywords"] + " " +
    df_combinado["cast"] + " " +
    df_combinado["crew"]
)

# Reemplazar espacios múltiples por un único espacio
df_combinado["tags"] = df_combinado["tags"].str.replace("\s+", " ", regex=True)

# Opcional: Reemplazar NaN por una cadena vacía en 'tags' si aún queda algún valor NaN
df_combinado["tags"].fillna("", inplace=True)



<ipython-input-18-f3fb6aa656b9>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combinado["tags"].fillna("", inplace=True)


In [ ]:
# Reemplazar NaN por una cadena vacía en 'tags' sin usar inplace=True
df_combinado["tags"] = df_combinado["tags"].fillna("")

In [ ]:
#En algún punto que me es difícil de precisar tengo la impresión de que las palabras
#no se están separando entre sí, lo cual no es útil para el modelo, que seguramente
#se creerá que todo el texto es una sola palabra muy larga.
#He probado con expresiones regulares, he probado a cargar cada dato por separado
#y analizar su estructura de forma independiente, he testeado si era cosa de los NaN,
#pero no estoy obteniendo ningún avance desde hace horas.
#Voy a reiniciar el planteamiento y lo voy a poner todo de la siguiente manera:
#Incluyo explicaciones de cada línea, porque al estar todo compactado en una es más
#difícil de recordar

In [ ]:
def preparar_json(json_str, default_value=None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value

#Empieza el código y esta vez (por fin) en una sola línea cada cosa, a asegurarse
#de que no hay ningún tipo de error que tenga que ver con strings vacíos/valores que
#no sean string y NaNs

df_combinado["genres"] = df_combinado["genres"].apply(lambda x: [item["name"] for item in preparar_json(x, [])] if pd.notna(x) and isinstance(x, str) else [])
df_combinado["keywords"] = df_combinado["keywords"].apply(lambda x: [item["name"] for item in preparar_json(x, [])] if pd.notna(x) and isinstance(x, str) else [])

#Lo mismo pero además se asegura de que cast no es una list
df_combinado["cast"] = df_combinado["cast"].apply(lambda x: [item["name"] for item in preparar_json(x, [])] if pd.notna(x) and isinstance(x, str) else [])[:3]
#Saca al nombre del director de la columna crew
df_combinado["crew"] = df_combinado["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in preparar_json(x, []) if crew_member.get('job', '').lower() == 'director']))
#Se asegura de que overview sea una lista de cadenas
df_combinado["overview"] = df_combinado["overview"].apply(lambda x: x if isinstance(x, list) else [str(x)])  # Handle non-list values

#Y ahora las combina todas en una nueva llamada tags
#A veces es un poco frustrante ver a la 1:56 lo increíblemente parecido que es este código
#de ahora, que funciona bien, y el de los primeros intentos que no funciona en absoluto

#En esta parte del código la diferencia es más clara y puedo ver que antes no me estaba asegurando de que
#isininstance estuviera presente para comprobar el tipo de objeto
df_combinado["tags"] = (
    df_combinado["overview"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x)) + " " +
    df_combinado["genres"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x)) + " " +
    df_combinado["keywords"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x)) + " " +
    df_combinado["cast"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x)) + " " +
    df_combinado["crew"]
)

#Limpio los tags por si acaso hubiera grupos de varios espacios de los que aparecen al
#concatenar dos partes en las que no había nada con espacios entre medias
df_combinado["tags"] = df_combinado["tags"].str.replace("\s+", " ", regex=True)
df_combinado["tags"] = df_combinado["tags"].fillna("")

In [39]:
#Ahora sí, llegamos a las recomendaciones. El Tfidvectorizer básicamente transforma
#los tags de antes en vectores numéricos para poder ver qué importancia tienen en
#el conjunto de datos
vectorizer = TfidfVectorizer()

#Esta es la matriz de filas-->películas y columnas-->palabras clave
tfidf_matrix = vectorizer.fit_transform(df_combinado["tags"])

#Y aquí se crea el modelo de vecinos más cercanos. El algoritmo que está usando es "brute"
#y la métrica es coseno (como ya había comentado antes). La siguiente línea es simplemente
#entrenarlo
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

#Y esta es la famosa función en la que se va a generar la recomendación. Es todo muy intuitivo
#Primero busca el índice de la película en el df por el título, luego calcula la distancia a
#las demás (como ya he explicado, cuanto más cerca más parecidas). El resto es llamar a la
#función e imprimir.
def get_movie_recommendations(movie_title):
    movie_index = df_combinado[df_combinado["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df_combinado["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "John Carter"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

#Un último detalle es que en el return similar_movies hay que poner[1:]
#Esto no es imprescindible pero lógicamente la película más similar a Avatar es Avatar
#Como esto pasa con todas, le pedimos que se salte la primera por similaridad, que es
#siempre la misma.

#Ahora voy a imprimir algunas recomendaciones para comprobar que efectivamente tiene
#sentido lo que está sacando.

#Avatar---> Apollo 18, Tears of the Sun, The American, The Inhabited Island, The Matrix
#Pirates of the Caribbean: At World's End--> What's Love Got to Do with It, My Blueberry Nights, Disturbia, Men in Black 3, A True Story
#John Carter--> Get Carter, The Marine 4: Moving Target, The Hurricane, Raising Cain, Mad Max: Fury Road

#Puedo ver el sentido de muchas de las recomendaciones, auque tampoco he visto todas
#las películas, se ve claramente que respeta la temática, estilo o presencia de ciertos actores etc
#Éxito!

Film recommendations 'John Carter'
- Film: Get Carter
- Film: The Marine 4: Moving Target
- Film: The Hurricane
- Film: Raising Cain
- Film: Mad Max: Fury Road
